In [33]:
import pandas as pd
import pandasql as sql
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
data = pd.read_csv('../data/total-ghg-emissions.csv')

## Original Dataset didn't include 2021-2022 for total_ghg

<p>Have used OWID as source again, where a dataset for Total greenhouse gas emissions" was downloaded, which contains data from 2021 and 2022.</p>
<p>"Annual greenhouse gas emissions by world region, 1850 to 2022
Greenhouse gas emissions include carbon dioxide, methane and nitrous oxide from all sources, including land-use
change. They are measured in tonnes of carbon dioxide-equivalents over a 100-year timescale."</p>

<p>Hannah Ritchie, Pablo Rosado and Max Roser (2020) - “Greenhouse gas emissions” Published online at OurWorldinData.org. Retrieved from: 'https://ourworldindata.org/greenhouse-gas-emissions' [Online Resource]</p>


In [34]:
import pandasql as sql

# Query with the total_ghg from all countries, setting aliases to be similar to the other dataset.
country_query = """
SELECT DISTINCT Code AS iso_code, Entity AS country, Year AS year,
                "Annual greenhouse gas emissions in CO₂ equivalents" AS total_ghg
FROM data
WHERE Code IS NOT NULL AND Code != "" AND LENGTH(Code) = 3
GROUP BY Code, Year
"""

country_result = sql.sqldf(country_query, locals())


In [35]:
# Map europe into north & south countries
northern_europe = [
    'NOR', 'SWE', 'FIN', 'DNK', 'ISL', 'GBR', 'IRL', 'DEU', 'NLD', 'POL', 
    'EST', 'LVA', 'LTU', 'RUS', 'BLR', 'CZE', 'SVK', 'FRA', 'UKR', 
    'CHE', 'AUT', 'HUN', 'BEL', 'LUX', 'MDA'
]

# Southern Europe with added countries: MCO (Monaco), SMR (San Marino), VAT (Vatican City), AND (Andorra), SRB (Serbia), MKD (North Macedonia)
southern_europe = [
    'ITA', 'ESP', 'PRT', 'GRC', 'MLT', 'CYP', 'HRV', 'SVN', 'BGR', 
    'ROU', 'ALB', 'BIH', 'MNE', 'MKD', 'MCO', 'SMR', 'VAT', 'AND', 'SRB'
]


# Add a 'north_or_south' column based on the north & south mapping.
country_result['north_or_south'] = country_result['iso_code'].apply(lambda x: 'Northern' if x in northern_europe else ('Southern' if x in southern_europe else None))

# Filter out to only grab countries with a north_or_south column.
europe_result = country_result[country_result['north_or_south'].notnull()]

In [36]:
print(europe_result)

      iso_code  country  year    total_ghg north_or_south
346        ALB  Albania  1850    1392864.4       Southern
347        ALB  Albania  1851    1403336.1       Southern
348        ALB  Albania  1852    1420242.0       Southern
349        ALB  Albania  1853    1432543.5       Southern
350        ALB  Albania  1854    1437972.4       Southern
...        ...      ...   ...          ...            ...
31827      UKR  Ukraine  2018  315419140.0       Northern
31828      UKR  Ukraine  2019  304761280.0       Northern
31829      UKR  Ukraine  2020  289791000.0       Northern
31830      UKR  Ukraine  2021  289413000.0       Northern
31831      UKR  Ukraine  2022  216495890.0       Northern

[7093 rows x 5 columns]


In [37]:
app = Dash(__name__)

# Dropdown country options
country_options = [{'label': country, 'value': country} for country in country_result['country'].unique()]

# Layout
app.layout = html.Div([
    html.H1("GHG Emissions Analysis by Country and Region"),
    
    html.Div([
        html.H2("Annual Greenhouse Gas Emissions Over Years by Country"),
        dcc.Dropdown(
            id='country-dropdown',
            options=country_options,
            value=country_options[0]['value'],
            clearable=False
        ),
        dcc.Graph(id='ghg-emission-trend-graph')
    ]),
    
    html.Div([
        html.H2("Total GHG Emissions by Country: Northern vs Southern Europe"),
        dcc.Graph(id='ghg-north-south-trend-graph')
    ])
])

@app.callback(
    Output('ghg-emission-trend-graph', 'figure'),
    Input('country-dropdown', 'value')
)
def update_annual_ghg_graph(selected_country):
    # Filter for selected country
    filtered_df = country_result[country_result['country'] == selected_country]

    fig = px.line(
        filtered_df,
        x='year',
        y='total_ghg',
        title=f"Total Greenhouse Gas Emissions for {selected_country}",
        labels={'year': 'Year', 'total_ghg': 'Total GHG Emissions (CO₂e)'}
    )
    return fig

@app.callback(
    Output('ghg-north-south-trend-graph', 'figure'),
    Input('ghg-north-south-trend-graph', 'id')
)
def update_north_south_ghg_graph(dummy_input):
    fig1 = px.line(
        europe_result,
        x='year',
        y='total_ghg',
        color='north_or_south',
        line_group='country',
        title="Total GHG Emissions by Country: Northern- vs Southern Europe",
        labels={'year': 'Year', 'total_ghg': 'Total GHG Emissions (CO₂e)'},
        color_discrete_map={'Northern': 'blue', 'Southern': 'red'}
    )
    fig1.update_traces(mode='lines', line=dict(width=1))
    fig1.update_xaxes(range=[1990, europe_result['year'].max()])

    return fig1

if __name__ == '__main__':
    app.run_server(debug=True, port=8089)




In [ ]:
app = Dash(__name__)

# Add a layout that has a year slider (from 1990 to 2023), subject to change.
app.layout = html.Div([
    dcc.Graph(id='choropleth-map'),
    dcc.Slider(
        id='year-slider',
        min=1990,
        max=2022,
        value=1990,
        marks={str(year): str(year) for year in range(1990, 2023)},
        step=1
    ),
    html.Div(id='selected-data')
])

@app.callback(
    Output('choropleth-map', 'figure'),
    Input('year-slider', 'value'))
def update_map(selected_year):
    # Filter the data to only show European countries.
    filtered_df = europe_result[europe_result['year'] == selected_year].copy()
    
    # Rename the columns to be more human-reading friendly.
    filtered_df = filtered_df.rename(columns={
        "total_ghg": "Total Greenhouse Gas Emission"
    })

    # Create the choropleth map with a custom color scale (gradient)
    fig = px.choropleth(
        filtered_df,
        locations="iso_code",
        color="Total Greenhouse Gas Emission",
        hover_name="country",
        color_continuous_scale=[
            # Start at a light yellow for the lowest emission, going to dark red for the highest. Should adjust the threshold.
            (0.0, "#ffffe0"),  # light yellow
            (0.2, "#ffd59b"),  # light orange
            (0.4, "#fdae61"),  # orange
            (0.6, "#f46d43"),  # red-orange
            (0.8, "#d73027"),  # red
            (1.0, "#a50026")   # dark red
        ],
        scope="world" # world map via plotly.
    )

    # Allow zoom, and center upon Europe on startup. Uses "natural earth" from Plotly.
    fig.update_geos(
        projection_type="natural earth",
        center={"lat": 50, "lon": 10},
        showcoastlines=True,
        coastlinecolor="Gray",
        showland=True,
        landcolor="lightgrey",
        showcountries=True
    )

    # Adds year, country name and title to the tooltip (cursor)
    fig.update_traces(
        hovertemplate=(
            "<b>%{hovertext}</b><br>"
            "Year: " + str(selected_year) + "<br>"
            "Total Greenhouse Gas Emission"
        )
    )
    fig.update_layout(
        coloraxis_colorbar=dict(
            title=f"Total Greenhouse Gas Emission in Year {selected_year}"
        )
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=7211)

